# TTRN em Neonatologia
Projeto 1 da Disciplina de Aprendizado de Máquina - PPgEEC/UFRN

Este projeto tem como objetivo desenvolver um *pipeline* para deposição de um modelo classificatório em um ambiente de produção.

O modelo se propõe a resolver um problema de **classificação** que tenta separar os recém nascidos que possam desenvolver uma patologia chamada **Taquipneia Transitória do Recém-Nascido(TTRN)** baseado nas seguintes variáveis(*labels*):

* Idade materna
* Fumo na gravidez
* Uso de álcool na gravidez
* Uso de drogas psicoativas na gravidez
* Trabalho de parto prematuro
* Doença Hipertensiva Específica da Gravidez(DHEG)
* Diabetes Mellitus Gestacional(DMG)
* Sexo do recém-nascido
* Índice de Apgar no quinto minuto de vida
* Reanimação neonatal
* Peso de nascimento
* Estatura ao nascimento
* Perímetro cefálico ao nascimento

Os dados para este modelo serão fornecidos pelo banco de dados da uma UTI neonatal do Estado após prévia autorização por escrito do responsável pelas informações.

## 1. Configuração do Ambiente de Trabalho

In [3]:
# Importações

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
import requests
import json

In [2]:
# Login no wandb
!wandb login c4d070b0b80b59891761b50eb9912749a4af585e --relogin

wandb: Appending key for api.wandb.ai to your netrc file: /Users/ericcalasans/.netrc


## 2.  Etapas do ETL

### 2.1. Aquisição dos Dados(*Fetch Data*)

Os dados são provenientes da plataforma https://neo.redcaprn.org através de API própria da plataforma

In [4]:
# Conexão com a base de dados e aquisição
data = {
    'token': '4276F33FF3B3F9E3CEA6A50EFAD0C213',
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'json'
}
r = requests.post('https://neo.redcaprn.org/api/',data=data)
r_dump = json.dumps(r.json())
df = pd.read_json(r_dump, dtype=True, convert_dates=True)

O *dataset* consiste de várias tabelas do qual são selecionadas as tabelas de interesse das quais serão escolhidos os *labels*:

In [5]:
ident = df[['nome_pac', 'nascido_hsc', 'setor_origem_pac', 'idade_materna',
            'cor_pele', 'proc_materna', 'escolaridade_materna', 'trabalho_mae',
            'gesta', 'abortos', 'filhos_vivos', 'natimortos', 'mortos_menor_1_ano',
            'ts_mae', 'cons_prenatal']]

ant_maternos = df[['fumo', 'alcool', 'freq_alcool', 'psicoativas', 'violencia',
                   'corioamnionite', 'rcui', 'tpp', 'dpp', 'has', 'dheg', 'hellp',
                   'sulfatada', 'oligoamnio', 'dm', 'hiv', 'hep_b', 'hep_c', 'sifilis',
                   'toxoplasmose', 'cmv', 'rubeola', 'covid_mae',
                   'outras_doencas_maternas', 'cort_antenatal']]

parto = df[['data_nasc', 'sexo', 'tempo_br', 'tipo_parto', 'gemelar',
            'apgar_1_minuto', 'apgar_5_min', 'reanimacao', 'o2_21',
            'massagem_cardiaca', 'cpap_sp', 'drogas_reanimacao', 'liq_amniotico',
            'clamp_cu', 'ev_hipotermia', 'saco_plastico', 'touca', 'colchao']]

antrop = df[['peso_nasc', 'estatura', 'pc', 'ig_nasc', 'ts_rn']]

Da tabela **resp** vem a variável *target* **ttrn**:

In [6]:
resp = df[['apneia', 'sdr', 'ttrn', 'sam', 'hpprn', 'enfisema', 'hem_pulm',
           'pneu_congenita', 'pneu_adquirida', 'dbp', 'o2circ', 'hood', 'cpap',
           'vni', 'vm', 'ox_nitrico', 'tempo_o2', 'o2_28', 'o2_36', 'num_intub',
           'falha_extub', 'surfactante', 'dose_surf', 'hv_surf', 'extub_acid']]

Formação do *dataset* bruto(*raw* dataset) através da seleção das *labels* e *target*:

In [15]:
ttrn_raw = pd.concat(
    [
        ident['idade_materna'],
        ant_maternos['fumo'],
        ant_maternos['alcool'],
        ant_maternos['psicoativas'],
        ant_maternos['tpp'],
        ant_maternos['dheg'],
        ant_maternos['dm'],
        parto['sexo'],
        parto['apgar_5_min'],
        parto['reanimacao'],
        antrop['peso_nasc'],
        antrop['estatura'],
        antrop['pc'],
        resp['ttrn'] # target
    ],
    axis=1
)

In [16]:
ttrn_raw.head()

,idade_materna,fumo,alcool,psicoativas,tpp,dheg,dm,sexo,apgar_5_min,reanimacao,peso_nasc,estatura,pc,ttrn
0,20,,,,2,2,1,2,8,1,3875,50,34,2
1,21,,,,2,2,1,1,4,4,810,33,25,2
2,16,,,,2,1,6,2,2,4,2240,51,32,2
3,40,,,,2,1,1,1,9,1,565,31,22,2
4,24,,,,2,1,1,2,9,1,1860,,,


Criação do artefato inicial

In [17]:
#  Salva como .csv
ttrn_raw.to_csv('ttrn_raw.csv', index=False)

In [20]:
# Cria o artefato
!wandb artifact put --name mlttrn/ttrn_raw.csv --type ttrn_raw --description "TTRN - Dados brutos" ttrn_raw.csv

wandb: Uploading file ttrn_raw.csv to: "ecalasans/mlttrn/ttrn_raw.csv:latest" (ttrn_raw)
wandb: Currently logged in as: ecalasans (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.15
wandb: Run data is saved locally in /Users/ericcalasans/Documents/Projetos/ml_proj1/wandb/run-20220513_042753-28lv6kck
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run glowing-jazz-1
wandb: ⭐️ View project at https://wandb.ai/ecalasans/mlttrn
wandb: 🚀 View run at https://wandb.ai/ecalasans/mlttrn/runs/28lv6kck
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("ecalasans/mlttrn/ttrn_raw.csv:latest")

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: Synced glowing-jazz-1: https://wandb.ai/ecalasans/mlttrn/run

### 2.2. Análise Exploratória(*EDA*)

### 2.3. Preprocessamento(*Preprocessing*)

### 2.4. Limpeza dos Dados(*Clean Data*)